In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv('C:/Users/Muxu/Downloads/MLOPS Assignment/Employee.csv')
data.head()

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4653 entries, 0 to 4652
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Education                  4653 non-null   object
 1   JoiningYear                4653 non-null   int64 
 2   City                       4653 non-null   object
 3   PaymentTier                4653 non-null   int64 
 4   Age                        4653 non-null   int64 
 5   Gender                     4653 non-null   object
 6   EverBenched                4653 non-null   object
 7   ExperienceInCurrentDomain  4653 non-null   int64 
 8   LeaveOrNot                 4653 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 327.3+ KB


In [4]:
data['LeaveOrNot'] = data['LeaveOrNot'].map({1:'Yes', 0:'No'})

In [5]:
X = data.drop(columns = ['LeaveOrNot'])
y = data['LeaveOrNot']

In [6]:
categorical_features = X.select_dtypes(include=[object]).columns

categorical_features= list(categorical_features.difference(['LeaveOrNot']))

print('\n','Categorical Features','\n', categorical_features,'\n')


 Categorical Features 
 ['City', 'Education', 'EverBenched', 'Gender'] 



In [7]:
numerical_features = list(X.select_dtypes(include=[np.float64,np.int64]))

print('\n','Numerical Features','\n', numerical_features,'\n')


 Numerical Features 
 ['JoiningYear', 'PaymentTier', 'Age', 'ExperienceInCurrentDomain'] 



In [8]:
from sklearn.model_selection import train_test_split

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
print('Train Data','\n',y_train.value_counts(normalize=True),'\n','\n','Test Data','\n', y_test.value_counts(normalize=True))

Train Data 
 LeaveOrNot
No     0.656368
Yes    0.343632
Name: proportion, dtype: float64 
 
 Test Data 
 LeaveOrNot
No     0.655209
Yes    0.344791
Name: proportion, dtype: float64


In [10]:
def summarize_cat(data,categorical_features):
  results=[]

  for column in data[categorical_features]:
      # Get the unique members of the column
      members = data[column].unique().tolist()
      # Append the column name and its unique members to the results list
      results.append([column, members])

  return pd.DataFrame(results, columns=['Column Name', 'Members'])

# Create a DataFrame from the results list
summarize_cat(X_train,categorical_features)

,Column Name,Members
0,City,"[New Delhi, Bangalore, Pune]"
1,Education,"[Masters, Bachelors, PHD]"
2,EverBenched,"[No, Yes]"
3,Gender,"[Male, Female]"


In [11]:
# EXPORTING FOR DE

my_feature_dict = {'CATEGORICAL' : summarize_cat(data,categorical_features).to_dict(), 'NUMERICAL' : {'Column Name': numerical_features}}

my_feature_dict.get('NUMERICAL')

{'Column Name': ['JoiningYear',
  'PaymentTier',
  'Age',
  'ExperienceInCurrentDomain']}

In [12]:
my_feature_dict

{'CATEGORICAL': {'Column Name': {0: 'City',
   1: 'Education',
   2: 'EverBenched',
   3: 'Gender'},
  'Members': {0: ['Bangalore', 'Pune', 'New Delhi'],
   1: ['Bachelors', 'Masters', 'PHD'],
   2: ['No', 'Yes'],
   3: ['Male', 'Female']}},
 'NUMERICAL': {'Column Name': ['JoiningYear',
   'PaymentTier',
   'Age',
   'ExperienceInCurrentDomain']}}

In [13]:
import pickle

# save dictionary to person_data.pkl file
with open('my_feature_dict.pkl', 'wb') as fp:
    pickle.dump(my_feature_dict, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


In [14]:
from sklearn.pipeline import Pipeline

# PREPROCESSING TRANSFORMATIONS ARE DONE ON EXAMPLE BASIS
# REAL WORLD SELECTION OF PREPROCSSING TRANSFORMATIONS MUST BE LOGICAL

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

pipeline_num = Pipeline(steps=[
    ('scale_data', StandardScaler()),
    ('simple_imputer1', SimpleImputer(strategy='constant',fill_value=0)),
])

from sklearn.preprocessing import OneHotEncoder

pipeline_cat = Pipeline(steps=[
    ('OneHotEncode', OneHotEncoder(handle_unknown="ignore"))
])

from sklearn.compose import ColumnTransformer

preprocessor_stage_1 = ColumnTransformer(
    transformers=[
        ('cat', pipeline_cat, categorical_features),  # Categorical columns
        ('num', pipeline_num, numerical_features),     # Numerical columns
    ],remainder='drop')

preprocessor_stack = Pipeline(steps=[
    ('preprocessor_stage_1', preprocessor_stage_1)
])

# BECAUSE WE DIDN'T SPECIFY CUSTOMERID IN ANY OF CATEGORICAL OR NUMERICAL FEATURES (REMAINDER='drop') REMOVE IT OUT OF PIPELINE

preprocessor_stack

Pipeline(steps=[('preprocessor_stage_1',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('OneHotEncode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['City', 'Education',
                                                   'EverBenched', 'Gender']),
                                                 ('num',
                                                  Pipeline(steps=[('scale_data',
                                                                   StandardScaler()),
                                                                  ('simple_imputer1',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant'))]),
                                                  ['JoiningYear', 'PaymentTier',
                                                   'Age',
                                                   'ExperienceInCurrentDomain'])]))])

In [15]:
preprocessor_stack.fit(X_train)

Pipeline(steps=[('preprocessor_stage_1',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('OneHotEncode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['City', 'Education',
                                                   'EverBenched', 'Gender']),
                                                 ('num',
                                                  Pipeline(steps=[('scale_data',
                                                                   StandardScaler()),
                                                                  ('simple_imputer1',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant'))]),
                                                  ['JoiningYear', 'PaymentTier',
                                                   'Age',
                                                   'ExperienceInCurrentDomain'])]))])

In [16]:
pd.DataFrame(preprocessor_stack.transform(X_train),columns=preprocessor_stack[-1].get_feature_names_out())

,cat__City_Bangalore,cat__City_New Delhi,cat__City_Pune,cat__Education_Bachelors,cat__Education_Masters,cat__Education_PHD,cat__EverBenched_No,cat__EverBenched_Yes,cat__Gender_Female,cat__Gender_Male,num__JoiningYear,num__PaymentTier,num__Age,num__ExperienceInCurrentDomain
0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,-1.109740,0.539424,0.117526,-1.861550
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,-1.645675,0.539424,-0.914641,0.061305
2,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.034001,-1.226396,-0.088907,-0.579646
3,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,-1.645675,0.539424,-1.121074,-0.579646
4,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.034001,-1.226396,1.149693,-0.579646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3717,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.573805,0.539424,0.530393,-1.220598
3718,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,-1.109740,0.539424,-0.708208,0.702257
3719,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.498065,0.539424,1.975427,-1.220598
3720,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.037870,0.539424,2.388294,-1.220598


In [17]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_stack),
    ('classifier', RandomForestClassifier())
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('preprocessor_stage_1',
                                  ColumnTransformer(transformers=[('cat',
                                                                   Pipeline(steps=[('OneHotEncode',
                                                                                    OneHotEncoder(handle_unknown='ignore'))]),
                                                                   ['City',
                                                                    'Education',
                                                                    'EverBenched',
                                                                    'Gender']),
                                                                  ('num',
                                                                   Pipeline(steps=[('scale_data',
                                                                                    StandardScaler()),
                                                                                   ('simple_imputer1',
                                                                                    SimpleImputer(fill_value=0,
                                                                                                  strategy='constant'))]),
                                                                   ['JoiningYear',
                                                                    'PaymentTier',
                                                                    'Age',
                                                                    'ExperienceInCurrentDomain'])]))])),
                ('classifier', RandomForestClassifier())])

In [18]:
# Checking Training Accuracy
y_train_pred = pipeline.predict(X_train)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_train,y_train_pred))
print("\nClassification Report:\n", classification_report(y_train,y_train_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_train,y_train_pred))

Accuracy: 0.9279957012358947

Classification Report:
               precision    recall  f1-score   support

          No       0.92      0.98      0.95      2443
         Yes       0.95      0.83      0.89      1279

    accuracy                           0.93      3722
   macro avg       0.93      0.91      0.92      3722
weighted avg       0.93      0.93      0.93      3722


Confusion Matrix:
 [[2387   56]
 [ 212 1067]]


In [19]:
# CREATING A TEST

my_pred_array=X_test.iloc[13:14:]
my_pred_array

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain
1610,Bachelors,2012,Bangalore,3,27,Male,Yes,5


In [20]:
pd.DataFrame(preprocessor_stack.transform(my_pred_array),columns=preprocessor_stack[0].get_feature_names_out())

,cat__City_Bangalore,cat__City_New Delhi,cat__City_Pune,cat__Education_Bachelors,cat__Education_Masters,cat__Education_PHD,cat__EverBenched_No,cat__EverBenched_Yes,cat__Gender_Female,cat__Gender_Male,num__JoiningYear,num__PaymentTier,num__Age,num__ExperienceInCurrentDomain
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,-1.645675,0.539424,-0.501774,1.343209


In [21]:
y_pred = pipeline.predict(my_pred_array)
y_pred

array(['No'], dtype=object)

In [22]:
y_test_pred = pipeline.predict(X_test)

# EVALUATE MODEL FOR TEST ACCURACY SINCE WE HAVE TEST SET
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_test_pred))

Accuracy: 0.8528464017185822

Classification Report:
               precision    recall  f1-score   support

          No       0.86      0.92      0.89       610
         Yes       0.83      0.72      0.77       321

    accuracy                           0.85       931
   macro avg       0.85      0.82      0.83       931
weighted avg       0.85      0.85      0.85       931


Confusion Matrix:
 [[563  47]
 [ 90 231]]


In [23]:
!pip install dill

In [24]:
import dill

# save trained pipeline file

with open('pipeline.pkl', 'wb') as file:
    dill.dump(pipeline, file)

print('pipeline saved successfully to file')

pipeline saved successfully to file


In [25]:
import dill

#Load the saved pipeline from the file

with open('pipeline.pkl', 'rb') as file:
    loaded_pipeline = dill.load(file)

print('pipeline loaded successfully to file')

pipeline loaded successfully to file


In [26]:
loaded_pipeline.__getstate__()

{'steps': [('preprocessor',
   Pipeline(steps=[('preprocessor_stage_1',
                    ColumnTransformer(transformers=[('cat',
                                                     Pipeline(steps=[('OneHotEncode',
                                                                      OneHotEncoder(handle_unknown='ignore'))]),
                                                     ['City', 'Education',
                                                      'EverBenched', 'Gender']),
                                                    ('num',
                                                     Pipeline(steps=[('scale_data',
                                                                      StandardScaler()),
                                                                     ('simple_imputer1',
                                                                      SimpleImputer(fill_value=0,
                                                                                    strategy=

In [27]:
y_pred = loaded_pipeline.predict(my_pred_array)

y_pred

array(['No'], dtype=object)